# Champions of sentiment discourse

AUTHOR: Michal Mochtak (michal.mochtak@ru.nl), Peter Rupnik (peter.rupnik@ijs.si), Nikola Ljubešić

DATE: 2024-06-24

---

In this notebook we look into specific countries and their sentiment scores on speaker- and party-level.

On the first run, the data will be downloaded from the internet. In the next cell a function was prepared to filter the dataset by specific conditions (e.g. taking only the MPs that have a specific number of speeches on the record). In the next cells we will inspect two countries in a comparable time frame, Croatia and France, and then the entire corpus across full time span.

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
from pathlib import Path
import seaborn as sns
from IPython.display import display
if not Path("speeches.csv.zip").exists():
    from os import system
    system("wget https://huggingface.co/datasets/5roop/parlasent_data/resolve/main/speeches.csv.zip")
df = pd.read_csv("speeches.csv.zip")

/tmp/ipykernel_3562638/258120034.py:9: DtypeWarning: Columns (10,11,12,13,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("speeches.csv.zip")


In [2]:
def calculate_sentiment(target="Speaker_name", *, country=None, term=None, filter=None):
    global df
    all_countries = df.country.unique().tolist()
    if country == None:
        country = input(f"Choose country from {all_countries} \n(empty for all): ")

    # Filtering
    # Select speeches from a specific country:
    c0 = df.country == country
    if country in ["all", ""]:
        c0 = pd.Series([True for i in df.country])
    # Keep only MPs
    c1 = (df.Speaker_MP == "MP")
    # Limit searches to speeches longer than 100 characters:
    c2 = df.char_length >= 100
    # Include only speakers with at least 10 speeches:
    gb = df[c0&c1&c2].groupby("Speaker_name").logits_pondered.count().reset_index()
    speakers_to_keep = gb.Speaker_name[gb.logits_pondered >= 10]
    c3 = df.Speaker_name.isin(speakers_to_keep)
    ndf = df[c0&c1&c2&c3]
    if term == None:
        print(f"Available terms:")
        display(ndf.groupby("Term").agg({
            "Date": [min, max, "count"],
        }).sort_values(("Date", "min")), clear=True, )
        term = input(f"Choose term from {ndf.Term.unique().tolist()} (empty for all): ")
    if term:
        c0 = ndf.Term == term
        nndf = ndf[c0].reset_index(drop=True)
    else:
        nndf = ndf
    def percentage_of_negative(l: pd.Series) -> float:
        return (l < 2.0).sum()/l.shape[0]
    gb2 = nndf.groupby([f"{target}"]).agg({
        "logits_pondered": [percentage_of_negative, "count"],
    }).reset_index()
    gb2.columns = f"{target} percentage_of_negative count".split()

    # gb2 = gb2.merge(gb1, on=f"{target}", how="left")
    gb2 = gb2.sort_values(by="percentage_of_negative", ascending=False)

    if filter:
        sigma = gb2["count"].std()
        mu = gb2["count"].median()
        q1, q3 = gb2["count"].quantile([0.25, 0.75]).values.tolist()
        if filter == "1sd":
            c = (gb2["count"] >= mu - sigma) & (gb2["count"] <= mu+sigma)
        elif filter == "2sd":
            c = (gb2["count"] >= mu - 2*sigma) & (gb2["count"] <= mu + 2*sigma)
        elif filter == "interquartile":
            c = (gb2["count"] >= q1) & (gb2["count"] <= q3)
        else:
            raise NotImplementedError("Only accepts '1sd' or '2sd' for now")
        gb2 = gb2[c]
    return gb2.reset_index(drop=True)

Let's inspect the terms we have available, so that an approximately equal timeframe can be set:

In [3]:
df[df.country.isin(["HR", "FR"])].groupby("country Term Speaker_MP".split()).agg({
    "Date": [min, max, "count"]
}).sort_values(by=("Date", "min"))

/tmp/ipykernel_3562638/2170860219.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df[df.country.isin(["HR", "FR"])].groupby("country Term Speaker_MP".split()).agg({
/tmp/ipykernel_3562638/2170860219.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df[df.country.isin(["HR", "FR"])].groupby("country Term Speaker_MP".split()).agg({


Date                    
                                           min         max   count
country Term            Speaker_MP                                
HR      5. mandat       MP          2003-12-22  2007-10-12   74856
                        notMP       2003-12-22  2007-10-12    4404
                        -           2004-04-01  2007-10-11    2632
        6. mandat       notMP       2008-01-11  2011-10-28    4200
                        MP          2008-01-11  2011-10-28   68561
                        -           2008-02-21  2011-10-27     455
        7. mandat       notMP       2011-12-22  2015-09-24    3824
                        MP          2011-12-22  2015-09-25   96544
                        -           2012-01-27  2015-07-03    1650
        8. mandat       MP          2015-12-03  2016-06-20   10559
                        notMP       2016-01-22  2016-06-20     477
                        -           2016-01-28  2016-03-02     110
        9. mandat       notMP       2016-10-14  2020-05-13    8002
                        MP          2016-10-14  2020-05-13  124817
                        -           2016-12-13  2020-02-05     791
FR      15e législature MP          2017-06-27  2022-03-23  413469
                        -           2017-06-27  2022-03-01   17765
                        notMP       2017-06-28  2022-03-23  283175
HR      10. mandat      MP          2020-07-22  2022-07-15   95051
                        notMP       2020-07-23  2022-07-15    5838
                        -           2020-12-17  2022-02-11    1563
FR      16e législature notMP       2022-06-28  2022-06-28      13
                        MP          2022-06-28  2022-06-29      17

In [4]:
calculate_sentiment("Speaker_name", country="HR", term="10. mandat")

,Speaker_name,percentage_of_negative,count
0,"Hasanbegović, Zlatko",0.961538,26
1,"Vidović Krišto, Karolina",0.950000,300
2,"Bernardić, Davor",0.949153,59
3,"Penava, Ivan",0.928571,28
4,"Mlinarić, Stipo",0.914110,163
5,"Spajić, Daniel",0.902778,72
6,"Raspudić, Nino",0.896266,482
7,"Prkačin, Ante",0.895349,86
8,"Petrov, Božo",0.892216,167
9,"Grmoja, Nikola",0.878113,763


In [5]:
calculate_sentiment("Speaker_name", country="FR", term="15e législature")

,Speaker_name,percentage_of_negative,count
0,"Parigi, Jean-François",0.928571,14
1,"Dupont-Aignan, Nicolas",0.926335,543
2,"Houplain, Myriane",0.923077,13
3,"Cornut-Gentille, François",0.857143,84
4,"Evrard, José",0.846154,26
5,"Corbière, Alexis",0.832402,1611
6,"Aubert, Julien",0.828783,1758
7,"Bilde, Bruno",0.826347,167
8,"Mélenchon, Jean-Luc",0.817950,1961
9,"Collard, Gilbert",0.814815,108


In [6]:
calculate_sentiment("Speaker_party", country="HR", term="10. mandat")


,Speaker_party,percentage_of_negative,count
0,HSS,0.861561,1358
1,SIP,0.833747,1209
2,MOST,0.812887,3554
3,DP,0.750384,1955
4,HS,0.678571,476
5,ZK,0.658354,2005
6,Fokus,0.625000,112
7,SDP,0.582735,4877
8,Pametno,0.578740,508
9,nezavisni,0.547862,12160


In [7]:
calculate_sentiment("Speaker_party", country="FR", term="15e législature")

,Speaker_party,percentage_of_negative,count
0,FI,0.781236,12982
1,GDR,0.694278,6519
2,UDI_I,0.659779,2710
3,-,0.641280,6250
4,NG,0.630577,1838
5,LT,0.602067,4355
6,LR,0.521784,27360
7,LC,0.517361,576
8,EDS,0.513178,645
9,UDI-A-I,0.492188,896


# Overall most negative and most positive parties



In [8]:
calculate_sentiment("Speaker_party_name", country="", term="",filter="1sd")

,Speaker_party_name,percentage_of_negative,count
0,Párbeszéd Magyarországért,1.000000,24
1,Srpska napredna stranka;Narodna seljačka stranka,1.000000,4
2,Crossbench;Labour,1.000000,1
3,Bündnis Zukunft Österreich,1.000000,5
4,Парламентарна група: Демократична България;Пар...,1.000000,1
5,Edustaja Väyrynen,1.000000,3
6,Jobbik Magyarországért Mozgalom,0.969697,33
7,Magyar Szolidaritás Mozgalom,0.962963,27
8,Respect,0.937500,16
9,PZDD,0.929078,141
